In [1]:
import mysql.connector
from mysql.connector import Error

In [3]:
!mysql.server start

Starting MySQL
 SUCCESS! 


In [6]:
def check_connection(host, db_name, user, psswd):
    try:
        connection = mysql.connector.connect(host=host,
                                             database=db_name,
                                             user=user,
                                             password=psswd)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)

    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [8]:
host = 'localhost'
db_name = 'sports_booking'
user = 'root'
psswd = ''
check_connection(host, db_name, user, psswd)

Connected to MySQL Server version  8.0.28
You're connected to database:  ('sports_booking',)
MySQL connection is closed


In [45]:
def execute_sql(query, host, db_name, user, psswd):
    try:
        connection = mysql.connector.connect(host=host,
                                             database=db_name,
                                             user=user,
                                             password=psswd)

        mySql_Create_Table_Query = query
        cursor = connection.cursor()
        result = cursor.execute(mySql_Create_Table_Query)
        print(result)

    except mysql.connector.Error as error:
        print("Failed to create table in MySQL: {}".format(error))
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            
def show_tables(host, db_name, user, psswd):
    try:
        connection = mysql.connector.connect(host=host,
                                             database=db_name,
                                             user=user,
                                             password=psswd)
        cursor = connection.cursor()
        cursor.execute("Show tables;")
        result = cursor.fetchall()
        for table in result:
            print(table)

    except mysql.connector.Error as error:
        print("Failed to create table in MySQL: {}".format(error))
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

## Creating the tables

In [30]:
create_members = """
CREATE TABLE members (
    id VARCHAR(255) PRIMARY KEY,
    password VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    member_since TIMESTAMP NOT NULL DEFAULT NOW(),
    payment_due FLOAT(4,2) NOT NULL DEFAULT 0
);
"""
execute_sql(create_members, host, db_name, user, psswd)

None
MySQL connection is closed


In [46]:
show_tables(host, db_name, user, psswd)

('members',)
MySQL connection is closed


In [47]:
create_pending_terminations = """
CREATE TABLE pending_terminations(
    id VARCHAR(255) PRIMARY KEY,
    email VARCHAR(255) NOT NULL,
    request_date TIMESTAMP NOT NULL DEFAULT NOW(),
    payment_due FLOAT(4,2) NOT NULL DEFAULT 0
    
);
"""
execute_sql(create_pending_terminations, host, db_name, user, psswd)

None
MySQL connection is closed


In [48]:
show_tables(host, db_name, user, psswd)

('members',)
('pending_terminations',)
MySQL connection is closed
